#### Export only Lviv records in csv
Radius of Lviv = 11 km

In [1]:
import numpy as np
import pandas as pd
import datetime
#import random
import os

In [2]:
# bigger dataset with q
path_to_sample_q = "/home/databox/data_sample_orig/bd_lab_large_sample_q.csv"

## smaller dataset from archive .gz
path_to_sample = "/home/databox/data_sample_orig/bd_lab_large_sample.csv"

In [3]:
# Column names -- from caption_file
headers = pd.read_csv("/home/databox/data_sample_orig/caption_file", sep = "\n", header = None,
                     squeeze = True)
headers = list(headers)
headers[:5]

['hash_number_A',
 'hash_tariff',
 'event_type',
 'event_sub',
 'network_service_direction']

##### Function `isInCity`

In [4]:
# Lviv center coordinates
center_lat, center_lon = 49.84, 24.03  # Lviv
#center_lat, center_lon = 48.28, 25.94  # Chernivtsi

# Radius of Lviv 
city_radius = 11 # km

## calculate length in km of 1 degree for latitude and longitude
a = 6378.137; b = 6356.752 # semiaxis of Earth ellipsoid in km
e2 = (a**2 - b**2) / a**2 # eccentricity of Earth

# Convert degree to radians
def deg_to_rad(phi):
    return np.pi * phi / 180

# one degree of latitude and longitude in km
deg_lat_in_km = np.pi / 180 * a * (1-e2) /\
            (1 - e2*np.sin(deg_to_rad(center_lat)))**1.5
deg_lon_in_km = np.pi / 180 * a * np.cos(deg_to_rad(center_lat)) /\
             np.sqrt(1 - e2*np.sin(deg_to_rad(center_lat))**2 )

print("1° of latitude = {0:5.2f} km".format(deg_lat_in_km)) 
print("1° of longitude = {0:5.2f} km".format(deg_lon_in_km))

#Convert distance in km to degrees
def km_to_lat(x):
    return x / deg_lat_in_km

def km_to_lon(x):
    return x / deg_lon_in_km


# check if point is inside circle with radius `city_radius` and center in (center_lat, center_lon)
def isInCity(lat, lon, center_lat, center_lon, city_radius):
    """True if (lat,lon) is in City, False otherwise"""
    return ((lat - center_lat)**2 / km_to_lat(city_radius)**2 +
            (lon - center_lon)**2 / km_to_lon(city_radius)**2) < 1


1° of latitude = 111.43 km
1° of longitude = 71.93 km


In [5]:
## MAIN
## Load and save dataframe by chunks

start_time = datetime.datetime.now()

file_Lviv = "bd_lab_large_Lviv.csv"

chunksize = 10**7
print("Chunksize = {0}".format(chunksize))
# delete previous Lviv_only.csv
try:
    os.remove(file_Lviv)
except:
    pass
for (i, chunk) in enumerate(pd.read_csv(path_to_sample_q,
                         chunksize=chunksize, na_values = '.', header = None, names = headers)):
    # Delete rows without latitude -- where LAT = '.'
    # Write to new dataframe df_with_locat
    
    chunk_with_locat = chunk[chunk['LAT'] != np.nan]

    # Convert latitude and longitude from `str` to `float`
    chunk_with_locat['LAT'] = chunk_with_locat['LAT'].astype(float)
    chunk_with_locat['LON'] = chunk_with_locat['LON'].astype(float)
    
    chunk_Lviv = chunk_with_locat[isInCity(chunk_with_locat['LAT'], chunk_with_locat['LON'], 
                                     center_lat, center_lon, city_radius)]
    print("chunk {0}: {1} records from Lviv".format(i, len(chunk_Lviv)))
    
    
    # to csv
    with open(file_Lviv, "a") as f:
        chunk_Lviv.to_csv(f, header = False)

finish_time = datetime.datetime.now()

print("\ntime = ".format(finish_time - start_time))

Chunksize = 10000000


/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


chunk 0: 452549 records from Lviv
chunk 1: 323275 records from Lviv
chunk 2: 343661 records from Lviv
chunk 3: 477141 records from Lviv
chunk 4: 529701 records from Lviv
chunk 5: 423000 records from Lviv
chunk 6: 436696 records from Lviv
chunk 7: 461367 records from Lviv
chunk 8: 382149 records from Lviv
chunk 9: 107743 records from Lviv

time = 


In [6]:
df_Lviv = pd.read_csv(file_Lviv, header = None, names = headers)
df_Lviv.to_csv(file_Lviv, index = False)

In [7]:
len(df_Lviv)

3937282

In [8]:
len(df_Lviv["hash_number_A"].unique())

19384

In [9]:
len(df_Lviv["hash_b_number"].unique())

194040

In [10]:
pd.set_option('display.max_columns', 50)

In [11]:
df_Lviv.head()

hash_number_A  hash_tariff   event_type  event_sub  \
10535         148731       239056  network_ser    content   
10536         148731       239056  network_ser    content   
10537         148731       239056  network_ser  omo_voice   
10538         148731       239056  network_ser    omo_sms   
10539         148731       239056  network_ser    omo_sms   

      network_service_direction     event_start_date  hash_b_number  \
10535                  Incoming   28AUG2017:00:53:31       188014.0   
10536                  Incoming   29AUG2017:12:58:36       188014.0   
10537                  Incoming   27AUG2017:19:57:30      1333643.0   
10538                  Incoming   28AUG2017:00:53:39      1335488.0   
10539                  Incoming   28AUG2017:00:53:48      1335488.0   

      number_B_category  cost  call_duration_minutes  data_volume_mb  \
10535                     0.0                    0.0             0.0   
10536                     0.0                    0.0             0.0   
10537                     0.0                    0.3             0.0   
10538                     0.0                    0.0             0.0   
10539                     0.0                    0.0             0.0   

       hash_accum_code       LAT        LON device_type  phone_price_category  \
10535              0.0  49.80861  24.017222                               1.0   
10536              0.0  49.82361  24.015000                               1.0   
10537              0.0  49.80861  24.017222                               1.0   
10538              0.0  49.80861  24.017222                               1.0   
10539              0.0  49.80861  24.017222                               1.0   

      interest_1 interest_2 interest_3 interest_4 interest_5  test_flag  \
10535                                                                 0   
10536                                                                 0   
10537                                                                 0   
10538                                                                 0   
10539                                                                 0   

       target  facebook_user  instagram_user  twitter_user  youtube_user  \
10535       1              0               0             0             0   
10536       1              0               0             0             0   
10537       1              0               0             0             0   
10538       1              0               0             0             0   
10539       1              0               0             0             0   

       viber_user  whatsapp_user  skype_user  
10535           0              0           0  
10536           0              0           0  
10537           0              0           0  
10538           0              0           0  
10539           0              0           0